In [3]:
from constants.abs_paths import AbsDirPath
from processing_pipeline.utilities.data_transformation import load_all_files

In [4]:
from processing_pipeline.data_optimization.stage_cfg import S0NoiseFiltering

current_stage_cfg = S0NoiseFiltering
current_stage_cfg

TypeError: StageConfig.__init__() missing 2 required positional arguments: 'out_dir' and 'stage_passed'

In [4]:
current_stage_cfg = current_stage_cfg.next_stage
current_stage_cfg

StageConfig(name='s3', depends_on_fields=['qa', 'sentence'], resulting_fields=['tactic', 'response'], data_dir=WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords/s3_tactic_extraction'), boolean_field_name=None, next_stage=None)

In [16]:
in_dir = current_stage_cfg.data_dir
out_path = AbsDirPath.SAMPLES / (current_stage_cfg.name + ".csv")
in_dir, out_path

(WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/keywords/s0_noise_filtering'),
 WindowsPath('C:/Users/Dmitrijs/Documents/myDocs/masters/courses/thesis/code/data/samples/s0.csv'))

In [ ]:
df = load_all_files(in_dir)
df.shape

Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\allenai.scispacy.v0.5.5.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\allenai.scispacy.v0.5.5.docs.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\allenai.scispacy.v0.5.5.issue.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\allenai.scispacy.v0.5.5.issue_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\allenai.scispacy.v0.5.5.release.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\google.deepvariant.v1.6.1.code_comment.parquet
Loaded C:\Users\Dmitrijs\Documents\myDocs\masters\courses\thesis\code\data\keywords\s0_noise_filtering\google.deepvariant.v1.6.1.

In [9]:
current_stage_cfg.get_columns()

['s3_tactic', 's3_response']

In [10]:
for col in current_stage_cfg.get_columns():
    try:
        df = df[df[col].str.len() > 0]
    except:
        pass
df.shape

(60229, 5)

In [11]:
sample_size = 384

In [12]:
out_path.parent.mkdir(exist_ok=True)
df.sample(sample_size).to_csv(out_path)
